In [13]:
print(__doc__)
from collections import Counter
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
pd.options.display.float_format = "{:.4f}".format
pd.set_option("display.max_rows", None, "display.max_columns", None)

Automatically created module for IPython interactive environment


In [14]:
def flatten(list):
    return [item for sublist in list for item in sublist]

def getSumOdds(list):
    return sum(list[1::2])

def getSumEvens(list):
    return sum(list[0::1])

In [15]:
path = r'/Users/wenxuanhuang/Documents/Repo/ML-for-COVID-19-dataset/result_log/af_test/EQODS_NF(200,10)'
all_files = sorted(glob.glob(os.path.join(path, "**/9.Result.csv")))

candidate_axis = []

for f in all_files:
    df = pd.read_csv(f)
    df = df.rename(columns={"# x_axis": "axis", " ur_accuracies": "ur_accuracies", " ur_fairness": "ur_fairness", " rs_accuracies": "rs_accuracies", " rs_fairness": "rs_fairness", " nonal_accuracies": "nonal_accuracies", " nonal_fairness": "nonal_fairness"})

    ur_a_min = df['ur_accuracies'].min()
    ur_a_max = df['ur_accuracies'].max()
    
    for idx in df.index:

        if df.iloc[idx]['ur_accuracies'] >= df.iloc[idx]['rs_accuracies'] or df.iloc[idx]['rs_accuracies'] - df.iloc[idx]['ur_accuracies'] <= (ur_a_max - ur_a_min)/3:
            if df.iloc[idx]['ur_fairness'] < df.iloc[idx]['rs_fairness']:
                candidate_axis = np.append(candidate_axis, df.iloc[idx]['axis'])


candidate_points = Counter(candidate_axis).most_common()

print(candidate_points)

file: /Users/wenxuanhuang/Documents/Repo/ML-for-COVID-19-dataset/result_log/af_test/EQODS_NF(200,10)/09021120_EQODS_NF(200,10)/9.Result.csv
candidate axis: [50.]
file: /Users/wenxuanhuang/Documents/Repo/ML-for-COVID-19-dataset/result_log/af_test/EQODS_NF(200,10)/09031540_EQODS_NF(200,10)/9.Result.csv
candidate axis: [50. 50.]
file: /Users/wenxuanhuang/Documents/Repo/ML-for-COVID-19-dataset/result_log/af_test/EQODS_NF(200,10)/09081220_EQODS_NF(200,10)/9.Result.csv
candidate axis: [50. 50. 40.]
file: /Users/wenxuanhuang/Documents/Repo/ML-for-COVID-19-dataset/result_log/af_test/EQODS_NF(200,10)/09141436_EQODS_NF(200,10)/9.Result.csv
candidate axis: [50. 50. 40. 40. 70.]
file: /Users/wenxuanhuang/Documents/Repo/ML-for-COVID-19-dataset/result_log/af_test/EQODS_NF(200,10)/09161209_EQODS_NF(200,10)/9.Result.csv
candidate axis: [ 50.  50.  40.  40.  70. 110. 130. 150. 160. 170.]
file: /Users/wenxuanhuang/Documents/Repo/ML-for-COVID-19-dataset/result_log/af_test/EQODS_NF(200,10)/09161214_EQODS_

In [16]:
sorted_frequency_arr = np.array(sorted(candidate_points, key=lambda x: x[0], reverse=False)).T[1]

sorted_frequency = sorted_frequency_arr.tolist()

print(sorted_frequency)


[8.0, 7.0, 7.0, 8.0, 7.0, 6.0, 2.0, 4.0, 3.0, 5.0, 4.0, 8.0, 6.0, 5.0, 6.0, 4.0, 3.0, 2.0, 5.0]
